In [ ]:
import tensorflow as tf
import numpy as np
from util.data import *

#### A simple FeedForward-Back Propagation NN is going to be implemented here. We are using toy data instead of using mnist.

##### Data and Parameters ---------------------------------------------------------------------------------------


In [ ]:
Sample_number=10000
Feature_Number=1024
batch_size=256
learning_rate=0.01
display_step=50
units_h=700 # number of units in hidden layer
units_o=10 # number of units in output layer which equals to the number of classes
epochs=1000

In [ ]:
DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)

#### structure definition ------------------------------------------------------------------------

In [ ]:
X=tf.placeholder(dtype=tf.float32,shape=[None,Feature_Number])
Y=tf.placeholder(dtype=tf.float32,shape=[None,10])

W_h=tf.Variable(tf.random_normal([Feature_Number,units_h],stddev=0.01))
b_h=tf.Variable(tf.random_normal([units_h]))

W_o=tf.Variable(tf.random_normal([units_h,units_o],stddev=0.01))
b_o=tf.Variable(tf.random_normal([units_o]))


def model(X,W_h,b_h,W_o,b_o):
    h=tf.nn.relu(tf.matmul(X,W_h)+b_h)
    o=tf.nn.sigmoid(tf.matmul(h,W_o)+b_o)
    result=tf.nn.softmax(o) # Note that since we are going to use softmax_cross_entropy_with_logits
    #in our cost function so, its not necesary to use the softmax here. It will not affect the cost function
    return result

Prediction=model(X,W_h,b_h,W_o,b_o)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Prediction,labels=Y))

train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted_label=tf.equal(tf.argmax(Prediction, 1), tf.argmax(Y, 1)) 
Accuracy=tf.reduce_mean(tf.cast(predicted_label, "float"))


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        avg_cost=0
        for j in range(Number_of_batches):
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            sess.run(train_step,feed_dict={X:Trx,Y:Try})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Try})/Number_of_batches
        if i% display_step==0:
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Train_x,Y:Train_y})
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y})
            
            print("Epoch: %03d/%03d cost: %.9f Train_Acc: %.3f"
                 %(i,epochs,avg_cost,Train_Accuracy,Test_Accuracy))
sess.close()
    